In [4]:
# exploratory_analysis.py (Revised for Stock Analysis)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set up visualization styles
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

def analyze_price_distribution(df: pd.DataFrame):
    """Analysis 1: Price Distribution per Category using Box Plots."""
    print("\n--- 1. Price Distribution per Category ---")
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Category', y='Price_Clean', data=df)
    plt.title('Product Price Distribution by Category')
    plt.xlabel('Category')
    plt.ylabel('Cleaned Price (USD)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show() 
    print("Insight: Box plots effectively summarize the median, spread, and outliers of product prices in each category.")


def analyze_rating_price_correlation(df: pd.DataFrame):
    """Analysis 2: Correlation between Rating and Price."""
    print("\n--- 2. Correlation between Rating and Price ---")

    correlation = df['Price_Clean'].corr(df['Rating'])

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='Price_Clean', y='Rating', data=df, alpha=0.6)
    plt.title(f'Rating vs. Price (Correlation: {correlation:.2f})')
    plt.xlabel('Cleaned Price (USD)')
    plt.ylabel('Rating')
    plt.tight_layout()
    plt.show()
    print(f"Insight: The correlation coefficient is {correlation:.2f}. Indicates the strength and direction of the linear relationship between price and rating.")


def analyze_top_reviewed_products(df: pd.DataFrame):
    """Analysis 3: Top Reviewed Products."""
    print("\n--- 3. Top Reviewed Products ---")

    reviewed_products = df[df['Reviews'] > 0]

    if not reviewed_products.empty:
        top_reviewed = reviewed_products.sort_values(by='Reviews', ascending=False).head(10)
        
        print("\nTop 10 Products by Review Count:")
        print(top_reviewed[['Name', 'Category', 'Reviews', 'Rating']])
        
        plt.figure(figsize=(12, 6))
        sns.barplot(x='Reviews', y='Name', data=top_reviewed, palette='viridis')
        plt.title('Top 10 Products by Total Reviews')
        plt.xlabel('Total Reviews')
        plt.ylabel('Product Name')
        plt.tight_layout()
        plt.show()
    else:
        print("Insight: Cannot visualize top reviews as the 'Reviews' column is currently filled with placeholders (0).")


def analyze_best_value_metric(df: pd.DataFrame):
    """Analysis 4: Best Value Metric (Rating-to-Price Ratio) per Category."""
    print("\n--- 4. Best Value Metric (Rating/Price) per Category ---")

    # Calculate Value Score: Rating / Price (filter non-positive values)
    df_value = df[(df['Price_Clean'] >= 1) & (df['Rating'] > 0)].copy()
    if df_value.empty:
        print("Insight: Cannot calculate Value Score, insufficient data with positive ratings and prices.")
        return

    df_value['Value_Score'] = df_value['Rating'] / df_value['Price_Clean']
    
    # Find the top product in each category by Value Score
    best_value_per_category = df_value.loc[df_value.groupby('Category')['Value_Score'].idxmax()]

    print("\nProduct with the Highest Value Score (Rating/Price) in Each Category:")
    print(best_value_per_category[['Category', 'Name', 'Rating', 'Price_Clean', 'Value_Score']].sort_values(by='Category'))
    print("Insight: Identifies products offering high customer satisfaction relative to their cost.")


def analyze_simulated_stock_availability(df: pd.DataFrame):
    """
    Analysis 5: Simulated Stock Availability based on assumed inventory trends.
    
    Since real stock data isn't available, we simulate a 'Low Stock Risk' score 
    based on the assumption that high reviews/demand and low price correlation 
    might lead to faster depletion.
    """
    print("\n--- 5. Simulated Stock Availability Analysis ---")

    # Create a simple 'Demand Proxy' based on Reviews (assuming high reviews = high demand)
    # Note: If Reviews are all 0, this will not be useful. We'll use Price Rank as a fallback.
    
    # Calculate Average Price Rank per Category
    avg_rank = df.groupby('Category')['Price_Category_Rank'].mean().sort_values(ascending=True)

    # We assume categories with products that are cheaper (lower Price Rank value) 
    # tend to sell out faster/have higher volume.
    
    # Visualize the proxy for volume/availability risk
    plt.figure(figsize=(10, 6))
    sns.barplot(x=avg_rank.index, y=avg_rank.values, palette='Reds_d')
    plt.title('Average Price Rank by Category (Proxy for Inventory Turnover)')
    plt.xlabel('Category')
    plt.ylabel('Average Price Rank (Lower value = Cheaper/Higher assumed volume)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print("\nCategories sorted by Average Price Rank (Lower Rank = Higher assumed turnover risk):")
    print(avg_rank)
    print("Insight: Categories with the lowest average Price Rank likely sell cheaper products, which might imply higher volume sales and potentially faster inventory turnover risk.")

# --- End of exploratory_analysis.py ---

--- Step 4: Loading Data to SQL Server ---
✅ CSV file 'my_new_scrape_data_CLEANED.csv' found.

Connecting to SQL Server (localhost)...
✅ Connected successfully.
✅ CSV Loaded and Cleaned: 91 rows found.

⏳ Inserting data into table...

❌ CRITICAL ERROR: Column Name Issue. The column 'Price_Category' was not found.
   Please check your CSV file again for correct column spelling.


In [12]:
data_list.append((
    ...
    row['Price_Category'], # <--- Yeh column CSV mein nahi mil raha
    ...
))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3679834875.py, line 2)

In [14]:
import pandas as pd
import pyodbc
import os
import warnings
warnings.filterwarnings('ignore') 

# --- Step 4: Loading Data to SQL Server ---
print("--- Step 4: Loading Data to SQL Server ---")

# Database Configuration
# ⚠️ Apne SQL Server ka Naam Zaroor Check Karein
SERVER = 'localhost' 
DATABASE = 'Banggood_Final'
DRIVER = 'ODBC Driver 17 for SQL Server' 

# Cleaned CSV file jo abhi taiyar hui hai
input_csv = "my_new_scrape_data_CLEANED.csv"

conn = None
df = pd.DataFrame()
csv_exists = False
connection_successful = False

# 1. Check CSV File
if not os.path.exists(input_csv):
    print(f"❌ Error: Cleaned CSV file '{input_csv}' not found.")
    print("   Pehle cleaning script run karein.")
    csv_exists = False
else:
    print(f"✅ CSV file '{input_csv}' found.")
    csv_exists = True

# --- Main Process ---
if csv_exists:
    try:
        # 2. Connect to Database
        print(f"\nConnecting to SQL Server ({SERVER})...")
        conn_str = f'DRIVER={{{DRIVER}}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        print("✅ Connected successfully.")
        connection_successful = True

        # 3. Load Data from CSV and Prepare
        df = pd.read_csv(input_csv)
        
        # Column names se extra spaces hatayein
        df.columns = df.columns.str.strip()
        
        # Convert NaN values to None for SQL NULLs (Zaroori step)
        df = df.where(pd.notnull(df), None)
        print(f"✅ CSV Loaded and Prepared: {len(df)} rows found.")

        # 4. Insert Data
        print("\n⏳ Inserting data into table...")
        
        # Purana data saaf karna (Agar zaroori ho)
        cursor.execute("TRUNCATE TABLE Products")
        
        # SQL INSERT QUERY (SQL table columns)
        # Note: Est_Revenue ko hum None (NULL) bhejenge.
        query = """
        INSERT INTO Products (Category, Name, Price, Rating, Reviews, URL, Price_Category, Est_Revenue)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        # Prepare data list (CSV column names)
        data_list = []
        for i, row in df.iterrows():
            data_list.append((
                row['Category'], 
                row['Name'], 
                row['Price_Clean'],          # CSV: Price_Clean -> SQL: Price
                row['Rating'], 
                row['Reviews'], 
                row['URL'], 
                row['Price_Category_Rank'],  # CSV: Price_Category_Rank -> SQL: Price_Category
                None                         # CSV mein Est_Revenue nahi hai -> SQL: NULL
            ))

        # Execute batch insert
        cursor.executemany(query, data_list)
        conn.commit() 
        print(f"✅ Upload complete. {len(data_list)} rows inserted.")

        # 5. Validate
        cursor.execute("SELECT COUNT(*) FROM Products")
        count = cursor.fetchone()[0]
        print(f"✅ Validation: Total rows in SQL Table = {count}")

        conn.close() 
        print("Database connection closed.")
        
    except pyodbc.Error as ex:
        # Connection ya SQL error hone par
        print(f"\n❌ SQL Connection or Insertion Error: {ex.args[1]}")
        if conn: conn.close()
            
    except Exception as e:
        # Koi aur galti hone par
        print(f"\n❌ General Error during process: {e}")
        if conn: conn.close()
            
else:
    print("\n⚠️ Process halted because CSV file was not found.")

--- Step 4: Loading Data to SQL Server ---
✅ CSV file 'my_new_scrape_data_CLEANED.csv' found.

Connecting to SQL Server (localhost)...
✅ Connected successfully.
✅ CSV Loaded and Prepared: 91 rows found.

⏳ Inserting data into table...
✅ Upload complete. 91 rows inserted.
✅ Validation: Total rows in SQL Table = 91
Database connection closed.
